In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction import stop_words
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
import re
import sys
import warnings
import pickle
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

In [2]:
dataset =  pd.read_csv("test.csv",encoding="latin1")
target = dataset["assignment_group"]
dataset= dataset.drop("assignment_group",axis=1)

In [3]:
dataset.head()

,Number,priority,description,assigned_to,state,created,closed,task_type,subcategory,close_code,closure_CI,cause_Code,CI_Specific_Code,closed_by,close_notes,type
0,INC0785295,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP BW(Business Warehouse))Redwood PRD: Jo...,NaN,New,7/19/2018 22:26,7/19/2018 23:37,Incident,ERP Misc,Solved (Permanently),SAP BW(Business Warehouse),Master Data,Incorrect/Missing Data,Deepak-kumar Arunkumar (dkarunku),issue:Job failure\r\nResoln:Repeat,2
1,INC0785718,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP BW(Business Warehouse))Redwood PRD: Jo...,Shiney Wilciya,InProgress,7/21/2018 16:56,7/23/2018 10:26,Incident,ERP Misc,Solved (Permanently),SAP BW(Business Warehouse),Job Failure,Job: Failure alert delay,Mohanraj Velusamy (mvelusam),Issue:(CI=SAP BW(Business Warehouse))Redwood P...,2
2,INC0787669,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP ERP FI (Finance)) JC_ZFICO_B_AP_CYC_AT...,NaN,New,7/29/2018 13:29,7/31/2018 12:31,Incident,Business Service,Solved (Permanently),SAP ERP Finance (Accounts Receivable),Job Failure,Job: Re-Execution,Sadhana Inuganti (sinugant),"bank issue, DNS error, rectified now",2
3,INC0786841,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP ERP FI (Finance)) JC_ZCRMZZ_BILLING DO...,Vrishali Mittal,InProgress,7/26/2018 2:54,7/31/2018 10:44,Incident,ERP Misc,Solved (Work Around),SAP Redwood,Job Failure,Job: Definition,Vinay kumar Balivada (vbalivad),The job is put on held . We will close the tic...,2
4,INC0785850,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP BW(Business Warehouse)) Redwood PRD: J...,NaN,New,7/22/2018 14:53,7/23/2018 10:27,Incident,ERP Misc,Solved (Permanently),SAP BW(Business Warehouse),Job Failure,Job: Failure alert delay,Mohanraj Velusamy (mvelusam),Issue:(CI=SAP BW(Business Warehouse))Redwood P...,2


In [4]:
list1 = []
for i in range(0,dataset.shape[0]):
    description = re.sub(r"\S*@\S*\s?","",dataset['description'][i])    
    description = re.sub("[^0-9a-zA-Z]"," ",description)
    description = description.lower()
    description = description.split()
    description = [word for word in description if not word in set(stop_words.ENGLISH_STOP_WORDS)]
    description = " ".join(sorted(set(description),key=description.index)) 
    description = re.sub(r"\b[a-zA-Z]\b","",description)
    list1.append(description)


In [5]:
tfIdf_vec = TfidfVectorizer()
tfidf_vectorizer= tfIdf_vec.fit_transform(list1)

In [6]:
index_value={i[1]:i[0] for i in tfIdf_vec.vocabulary_.items()}

In [7]:
tfidf_vectorizer

<22x55 sparse matrix of type '<class 'numpy.float64'>'
	with 292 stored elements in Compressed Sparse Row format>

In [8]:
fully_indexed = []
for row in tfidf_vectorizer:
    fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

In [9]:
tfIdf_vec.vocabulary_.items()

dict_items([('ci', 22), ('sap', 48), ('bw', 20), ('business', 19), ('warehouse', 52), ('redwood', 45), ('prd', 42), ('job', 35), ('ended', 28), ('ok', 39), ('erp', 30), ('fi', 32), ('finance', 33), ('jc', 34), ('zfico', 54), ('ap', 13), ('cyc', 24), ('atlut', 15), ('59666659', 6), ('status', 49), ('error', 31), ('zcrmzz', 53), ('billing', 17), ('doc', 27), ('step7', 50), ('59530391', 1), ('otc', 40), ('prfar002c', 43), ('59434256', 0), ('killed', 36), ('59661673', 2), ('b1', 16), ('dly', 26), ('prb2b002c', 41), ('non', 37), ('59661675', 3), ('59665221', 5), ('59665219', 4), ('59666661', 7), ('59669644', 9), ('59669642', 8), ('59671824', 11), ('59671822', 10), ('59672354', 12), ('bpc', 18), ('nw', 38), ('epm', 29), ('cost', 23), ('centre', 21), ('reporting', 47), ('atl', 14), ('transaction', 51), ('data', 25), ('refresh', 46), ('process', 44)])

In [10]:
y = pd.get_dummies(target)

In [11]:
tfIdf_vec.vocabulary_
vocab_dict={}
for key,value in enumerate(tfIdf_vec.vocabulary_):
    vocab_dict[key] = value

In [13]:
#vocab_dict

In [14]:
from sklearn.linear_model import LogisticRegression
lr =  LogisticRegression()

model = lr.fit(tfidf_vectorizer,target)

c:\users\deepak.saini\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\deepak.saini\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [16]:
#model.classes_  #gives the classes associated with model
model.coef_   #feature importance per class 

array([[-0.04805284, -0.05077897, -0.05168511, -0.04609648, -0.05168511,
        -0.04609648, -0.05168511, -0.04609648, -0.05168511, -0.04609648,
        -0.05168511, -0.04609648, -0.04609648, -0.25653184,  0.20544691,
        -0.26601546, -0.17592436, -0.05077897,  0.20544691, -0.29633219,
        -0.29633219,  0.20544691, -0.27733178,  0.20544691, -0.19494262,
         0.20544691, -0.17592436, -0.05077897, -0.27733178,  0.20544691,
        -0.27553755, -0.26725924, -0.27553755, -0.27553755, -0.19667664,
        -0.30581033, -0.04805284, -0.17592436,  0.20544691, -0.17083208,
        -0.04805284, -0.17592436, -0.17083208, -0.04805284,  0.20544691,
        -0.17083208,  0.20544691,  0.20544691, -0.27733178, -0.27553755,
        -0.05077897,  0.20544691, -0.29633219, -0.05077897, -0.26601546],
       [-0.0966861 , -0.08921641, -0.08371991, -0.07353718, -0.08371991,
        -0.07353718, -0.08371991, -0.07353718, -0.08371991, -0.07353718,
        -0.08371991, -0.07353718, -0.07353718, -0.

In [92]:
outer_dict_lr={}
inner_dict_lr={}
for i in range(len(model.classes_)):
    for j in range(len(model.coef_[i])):
        inner_dict_lr[vocab_dict[j]]=model.coef_[i][j]
    outer_dict_lr[model.classes_[i]] = inner_dict

#outer_dict_lr

{'SAP BPC Support': {'ci': 0.12366577592874546,
  'sap': -0.20596461001454364,
  'bw': 0.11339586483824052,
  'business': 0.09415775463610056,
  'warehouse': 0.11339586483824052,
  'redwood': 0.09415775463610056,
  'prd': 0.11339586483824052,
  'job': 0.09415775463610056,
  'ended': 0.11339586483824052,
  'ok': 0.09415775463610056,
  'erp': 0.11339586483824052,
  'fi': 0.09415775463610056,
  'finance': 0.09415775463610056,
  'jc': 0.5427525807017958,
  'zfico': -0.10621915924450512,
  'ap': 0.572854205380612,
  'cyc': 0.35934724280646463,
  'atlut': -0.20596461001454364,
  '59666659': -0.10621915924450512,
  'status': -0.5885036303238182,
  'error': -0.5885036303238182,
  'zcrmzz': -0.10621915924450512,
  'billing': -0.0295637142351043,
  'doc': -0.10621915924450512,
  'step7': 0.4393007130162668,
  '59530391': -0.10621915924450512,
  'otc': 0.35934724280646463,
  'prfar002c': -0.20596461001454364,
  '59434256': -0.0295637142351043,
  'killed': -0.10621915924450512,
  '59661673': 0.456

In [115]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model_rf = rf.fit(tfidf_vectorizer,target)
model_rf.feature_importances_

inner_dict_rf={}
for j in range(len(model_rf.feature_importances_)):
    inner_dict_rf[vocab_dict[j]]=model_rf.feature_importances_[j]




C:\Users\deepak.saini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [94]:
test = pd.read_csv("Book2.csv")

In [95]:
test.head()

,Number,priority,description,Unnamed: 3,assigned_to,state,created,closed,task_type,subcategory,close_code,closure_CI,cause_Code,CI_Specific_Code,closed_by,close_notes,type
0,INC0785295,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP BW(Business Warehouse))Redwood PRD: Jo...,NaN,NaN,New,7/19/2018 22:26,7/19/2018 23:37,Incident,ERP Misc,Solved (Permanently),SAP BW(Business Warehouse),Master Data,Incorrect/Missing Data,Deepak-kumar Arunkumar (dkarunku),issue:Job failure\r\nResoln:Repeat,2
1,INC0785718,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP BW(Business Warehouse))Redwood PRD: Jo...,NaN,Shiney Wilciya,InProgress,7/21/2018 16:56,7/23/2018 10:26,Incident,ERP Misc,Solved (Permanently),SAP BW(Business Warehouse),Job Failure,Job: Failure alert delay,Mohanraj Velusamy (mvelusam),Issue:(CI=SAP BW(Business Warehouse))Redwood P...,2
2,INC0787669,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP ERP FI (Finance)) JC_ZFICO_B_AP_CYC_AT...,NaN,NaN,New,7/29/2018 13:29,7/31/2018 12:31,Incident,Business Service,Solved (Permanently),SAP ERP Finance (Accounts Receivable),Job Failure,Job: Re-Execution,Sadhana Inuganti (sinugant),"bank issue, DNS error, rectified now",2
3,INC0786841,4 - Low (Resolution SLA ? 5 Business days),(CI=SAP ERP FI (Finance)) JC_ZCRMZZ_BILLING DO...,NaN,Vrishali Mittal,InProgress,7/26/2018 2:54,7/31/2018 10:44,Incident,ERP Misc,Solved (Work Around),SAP Redwood,Job Failure,Job: Definition,Vinay kumar Balivada (vbalivad),The job is put on held . We will close the tic...,2


In [96]:
list2 = []
for i in range(0,test.shape[0]):
    desc = re.sub(r"\S*@\S*\s?","",test['description'][i])    
    desc = re.sub("[^0-9a-zA-Z]"," ",desc)
    desc = desc.lower()
    desc = desc.split()
    desc = [word for word in desc if not word in set(stop_words.ENGLISH_STOP_WORDS)]
    desc = " ".join(sorted(set(desc),key=desc.index)) 
    desc = re.sub(r"\b[a-zA-Z]\b","",desc)
    list2.append(desc)


In [97]:
test_vectorizer= tfIdf_vec.transform(list2)
#tfidf_vectorizer

In [78]:
test_vectorizer

<4x55 sparse matrix of type '<class 'numpy.float64'>'
	with 48 stored elements in Compressed Sparse Row format>

In [114]:
#model.predict(test_vectorizer)
model_rf.predict(test_vectorizer)

array(['SAP BW Support', 'SAP BW Support', 'SAP FICO Support',
       'SAP Basis Support'], dtype=object)

In [99]:
to_predict_vocab ={}
for i in range(0,test.shape[0]):
    desc = re.sub(r"\S*@\S*\s?","",test['description'][i])    
    desc = re.sub("[^0-9a-zA-Z]"," ",desc)
    desc = desc.lower()
    desc = desc.split()
    desc = [word for word in desc if not word in set(stop_words.ENGLISH_STOP_WORDS)]
    desc = " ".join(sorted(set(desc),key=desc.index)) 
    desc = re.sub(r"\b[a-zA-Z]\b","",desc)
    
    to_predict_vocab[i] = desc
to_predict_vocab

{0: 'ci sap bw business warehouse redwood prd job ended ok',
 1: 'ci sap bw business warehouse redwood prd job ended ok',
 2: 'ci sap erp fi finance jc zfico  ap cyc atlut 1 59666659 ended status error',
 3: 'ci sap erp fi finance jc zcrmzz billing doc step7 59530391 ended status error'}

In [119]:
inner_dict_rf

#other than forest , we get the dictionary of feature's with respect to diffrent categories
#predicted_vocab_dict= outer_dict_lr["SAP BPC Support"]
for i in range(len(to_predict_vocab.items())):
    temp=[]
    tmp_list=[]
    tmp_list=to_predict_vocab[i].split()
    for j in tmp_list:
        if j in inner_dict_rf.keys():
            temp.append(inner_dict_rf[j])
            #print(temp)
    
    temp = sorted(temp,reverse=True)
    print(i,temp)        
    
    

0 [0.007801418439716314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1 [0.007801418439716314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2 [0.01698841698841699, 0.007801418439716314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3 [0.12957891591175233, 0.022184873949579836, 0.01698841698841699, 0.007801418439716314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [116]:
#if the predicted grp is SAP BPC Support

predicted_vocab_dict= outer_dict_lr["SAP BPC Support"]



{'SAP BPC Support': {'ci': 0.12366577592874546,
  'sap': -0.20596461001454364,
  'bw': 0.11339586483824052,
  'business': 0.09415775463610056,
  'warehouse': 0.11339586483824052,
  'redwood': 0.09415775463610056,
  'prd': 0.11339586483824052,
  'job': 0.09415775463610056,
  'ended': 0.11339586483824052,
  'ok': 0.09415775463610056,
  'erp': 0.11339586483824052,
  'fi': 0.09415775463610056,
  'finance': 0.09415775463610056,
  'jc': 0.5427525807017958,
  'zfico': -0.10621915924450512,
  'ap': 0.572854205380612,
  'cyc': 0.35934724280646463,
  'atlut': -0.20596461001454364,
  '59666659': -0.10621915924450512,
  'status': -0.5885036303238182,
  'error': -0.5885036303238182,
  'zcrmzz': -0.10621915924450512,
  'billing': -0.0295637142351043,
  'doc': -0.10621915924450512,
  'step7': 0.4393007130162668,
  '59530391': -0.10621915924450512,
  'otc': 0.35934724280646463,
  'prfar002c': -0.20596461001454364,
  '59434256': -0.0295637142351043,
  'killed': -0.10621915924450512,
  '59661673': 0.456